# 基于用户的协同过滤算法 `userCF`

In [1]:
import random
import math
import time
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import KFold

## 一、通用函数定义

In [2]:
# 定义装饰器，监控运行时间
def timmer(func):
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        res = func(*args, **kwargs)
        stop_time = time.perf_counter()
        print("Func {}, run time: {}".format(func.__name__, stop_time - start_time))
        return res
    return wrapper

### 1. 数据处理
- `load data`
- `split data`

数据格式（有换行）： `[user_id ::  movie_id :: rating :: timestamp]`

        1::1193::5::978300760

        1::661::3::978302109
        
- 基于用户或item的协同过滤只需要保留<font color=red> user </font>和 <font color=red> item </font>字段数据即可。处理数据时，需要将换行符 `'\n'`和 分隔符`'::'`去掉;
- 需求数据格式 tuple(user, item), tuple元素为int

In [3]:
class Dataset():
    def __init__(self, filepath):
        self.data = self.loadData(filepath)
        
    @timmer
    def loadData(self, filepath):
        '''返回值data: 元组(user, item)为元素的列表'''
        data = []
        for line in open(filepath):
            temp = tuple(map(int, line.strip().split("::")[:2]))   # index 0和1是 user和item
            data.append(temp)
#         print('部分原始数据: {}'.format(data[:10]), type(data))
        return data
    
    @timmer
    def splitData(self, kfolds=5, seed=1):
        '''
        :params: data, 格式元组(user, item)
        :params: kfolds, k折交叉验证的n_split 
        :params: seed, random的种子数
        '''
#         # K折交叉验证获取train和test
#         kf = KFold(n_splits=kfolds)
#         for train_index, test_index in kf.split(self.data):
#             train = np.array(self.data)[train_index]
#             test = np.array(self.data)[test_index]
#             break
#         print(train.shape, test.shape)    # (800168, 2)
        train, test = [], []
        random.seed(seed)
        
        for user, item in self.data:
            # 这里与书中的不一致，本人认为取M-1较为合理，因randint是左右都覆盖的
            if random.randint(0, kfolds-1) == 1:  
                test.append((user, item))
            else:
                train.append((user, item))
                
#         print(len(train), len(test))
#         print(train[5800:5810], test[100:110])
            
        def convert_dict(data):
            '''转换成字典形式 {user: set(items)}'''
#             assert(type(data) == np.ndarray)
#             data = data.tolist()  # ndarrya 数组转化为list 需要使用 arr.tolist()函数
            data_dict = {}
            for user, item in data:
                if user not in data_dict:
                    data_dict[user] = set()
                data_dict[user].add(item)     # 集合添加元素的方法 .add(ele)
#             data_dict转化为 user 为key, set(item)转为 list 为value的字典（可要可不要）
#             for key in data_dict.keys():
#                 data_dict[key] = list(data_dict[key])
#             print('~~~~~~~~~~~~', data_dict)
            return data_dict
        return convert_dict(train), convert_dict(test)       

训练集和测试机数据格式 ： 
>`{user1: [item11, item12, ...], user2:[item21, item22,...]}`

In [4]:
if __name__ == "__main__":
    filepath = r'..\data\ml-1m\ratings.dat'
    dataset = Dataset(filepath)
    dataset.splitData(5, 2)

Func loadData, run time: 1.4070679909999995
Func splitData, run time: 1.5393819129999997


### 2. 评价指标
- Precision
- Recall
- Coverage
- Popularity(Novelty)

In [5]:
class Metric():
    def __init__(self, train, test, GetRecommendation):
        '''
        :params: train, 训练数据
        :params: test, 测试数据
        :params: GetRecommendation, 为某个用户获取推荐物品的接口函数
        '''
        self.train = train
        self.test = test
        self.GetRecommendation = GetRecommendation
        self.recs = self.getRec()
        
    def getRec(self):
        '''为 test 中的每个用户进行推荐'''
        recs = {}
        for user in self.test.keys():
            rank = self.GetRecommendation(user)  # rank为列表
            recs[user] = rank
        return recs
    
    def precision(self):
        '''准确率：最终的推荐列表中有多少比例是 **发生过的用户-物品行为记录** '''
        All, hit = 0, 0
        for user in self.test.keys():
            # 用户在test中喜欢的item 集合 T(u)
            test_items = set(self.test[user])
            # 对用户推荐的N个item 列表 （rank为列表）
            rank = self.recs[user]
            for item, score in rank:
                if item in test_items:
                    hit += 1               # 分子： sum(T(u) & R(u)))
            All += len(rank)               # 分母： sum(R(u))
            # precision = (T(u) & R(u)) / R(u)
        return round(hit / All * 100, 2)
        
    def recall(self):
        '''召回率：有多少比例的 **用户-物品行为记录** 包含在最终的推荐列表中'''
        All, hit = 0, 0
        for user in self.test.keys():
            # 用户在 test 中喜欢的 item 集合 T(u)
            test_items = set(self.test[user])
            # 对用户推荐的N个 item 列表R(u)
            rank = self.recs[user]
            for item, score in rank:
                if item in test_items:
                    hit += 1               # 分子： sum(T(u) & R(u)))
            All += len(test_items)         # 分母： sum(T(u))
            # recall = (T(u) & R(u)) / T(u)
        return round(hit / All * 100, 2)
    
    def coverage(self):
        '''覆盖率：最终的推荐列表中包含多大比例的 **物品**'''
        all_item, recom_item = set(), set()
        for user in self.test.keys():         # test 中的 user
            for item in self.train[user]:     # test中user在train中包含的所有item
                # 凡是train中user有过行为记录的item都加入到all_item中
                all_item.add(item)            # 所有物品集合 I
            # 对用户推荐的N个item 列表
            rank = self.recs[user]
            # 凡是推荐给user的item都计入到recom_item中
            for item, score in rank:
                recom_item.add(item)          # 推荐的物品集合 R(u)
        # coverage = #R(u) / #I
        return round(len(recom_item) / len(all_item) * 100, 2)
    
    def popularity(self):
        '''新颖度：推荐的是目标用户喜欢的但未发生过的用户-行为
           用推荐列表中物品的平均流行度来度量新颖度(新颖度越高，流行度越低)'''
        # 计算item 的流行度 （train set）
        item_popularity = {}
        for user, items in self.train.items():
            for item in items:
                if item not in item_popularity:
                    item_popularity[item] = 0
                # 若item在train的user中发生过记录，则该item的流行度+1
                item_popularity[item] += 1
        
        popular = 0
        num = 0
        for user in self.test.keys():
            # 向test中的 user 推荐topN 物品
            rank = self.recs[user]
            for item, score in rank:
                # 对每个物品的流行度取对数运算
                # 防止长尾问题带来的北流行物品主导的推荐（避免热门推荐）
                popular += math.log(1 + item_popularity[item])
                # 汇总所有user的总推荐物品个数
                num += 1
        # 计算平均流行度 = popular / n
        return round(popular / num, 6)
    
    def eval(self):
        # 汇总 metric 指标
        metric = {'Precision': self.precision(),
                  'Recall': self.recall(),
                  'Coverage': self.coverage(),
                  'Popularity': self.popularity()}
        print('Metric: {}'.format(metric))
        return metric

## 二、算法实现
- **Random**： 随机推荐N个用户 *未见过* 的item
- **MostPopular**：随机推荐N个用户*未见过* 的*最热门*的item
- **UserCF**
- **UserIIF**

### 1. Random 随机推荐
- 随机推荐 N 个 用户未见过的 item

In [6]:
# 1. 随机推荐
def Random(train, K, N):
    '''
    :params: train, 训练数据集
    :params: K, 可忽略
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation, 推荐接口函数
    '''
    items = {}
    for user in train.keys():
        for item in train[user]:
            items[item] = 1
    
    def GetRecommendation(user):
        '''根据items字典中的item出现次数为依据，随机选取topN个未见过的item作为推荐内容'''
        user_items = set(train[user])   # 目标用户user的列表中的item集合（推荐的item不应与该集合中的item相同）
        recom_items = {}                # 定义推荐列表，字典（item, #item）
        for item in items.keys():       # items 前文定义的，包含所有 item 的字典(item, #item)
            if item not in user_items:
                recom_items[item] = items[item]  # 未见过的item添加为recom_items的元素
        # 从recom_items中随机挑选 N个 ： [(item1, #item1), (item2, #item2), ...]
        recom_items = list(recom_items.items())
        random.shuffle(recom_items)
        return recom_items[: N]
    
    return GetRecommendation

### 2. MostPopular 热门推荐
- 随机推荐 N 个 用户未见过的 最热门的 item

In [7]:
def MostPopular(train, K, N):
    '''
    :params: train, 训练数据集
    :params: K, 可忽略
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation, 推荐接口函数
    '''
    items = {}                   # keys: item, value: item出现的次数
    for user in train.keys():
        for item in train[user]:
            if item not in items.keys():
                items[item] = 0  # 若果是新item, 先设置item出现的次数为0，再+1计数
            items[item] += 1     # user中出现一次item 则+1计数一次
    
    def GetRecommendation(user):
        '''根据items字典中的item出现次数为依据，随机选取topN个未见过的item作为推荐内容'''
        user_items = set(train[user])   # 目标用户user的列表中的item集合（推荐的item不应与该集合中的item相同）
        recom_items = {}                # 定义推荐列表，字典（item, #item）
        for item in items.keys():       # items 前文定义的，包含所有 item 的字典(item, #item)
            if item not in user_items:
                recom_items[item] = items[item]  # 未见过的item及出现的次数添加为recom_items的元素
        # 从recom_items中挑选 topN ： [(item1, #item1), (item2, #item2), ...]
        recom_items_sorted = sorted(recom_items.items(), key=lambda x: x[1], reverse=True)
        '''若只保存 item，不需要#item: [item1, item2, item3, ...]'''
#         recom_items_final = list(map(lambda x: x[0], recom_items_sorted))
        return recom_items_sorted[: N]
    
    return GetRecommendation

### 3. UserCF 基于用户的协同过滤推荐 （余弦相似度）
步骤：
1. 计算 <font color=blue>item --> user</font> 的倒排表
    - 每个物品都保存对该物品产生过行为的用户列表 <font color=blue>`dict{item, set(user1, user4, user7), ...}`</font> 
2. 计算每两个用户之间的共同item情况的稀疏矩阵Cmat[u][v]-(可用dict保存)
    - 比如，对于不同的item,如A、B和C，用户u和用户v的item列表中都有，则`Cmat[u][v]=3`）
    - 比如，对于不同的item,如A、B和C，用户u和用户v的item列表中都有A和B，不同时存在C，则`Cmat[u][v]=2`）
    - 具体的 物品--用户倒排表见book《系统推荐实践》P47
3. 根据2中用户之间关于co-rated item的稀疏矩阵，计算用户之间的余弦相似度
4. 按照相似度进行排序兴趣相似的K个用户
5. 将K个用户喜欢的物品计算rank值并推荐给user 
    - 此例权重皆设为1，电影中可以将用户对电影的评分作为基础，用户相似性得分作为权重
    - 权重为1，则直接将相似度作为用户user对物品i感兴趣的程度，将相似度传给rank值作为推荐topN的标准

余弦相似度公式中:
- N(user): 用户user正经有过正反馈的物品集合的元素总数
        - N(u)
        - N(v)

In [8]:
# 3. 基于用户余弦相似度的推荐
def UserCF(train, K, N):
    '''
    :params: train, 训练数据集
    :params: K, 超参数，设置取TopK相似用户数目
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation, 推荐接口函数
    '''
    # 计算 item->user 的倒排表 {item, set(user1, user4, user7), ...}
    item_users = {}
    for user in train.keys():             # 遍历用户
        for item in train[user]:          # 遍历该循环内user对应的所有items
            if item not in item_users.keys():
                item_users[item] = set()  # item_users新增的item，先设置对应user集合为空集合
            item_users[item].add(user) # 设置为空集合后，将该新增的item对应的user添加到用户集合中
    
    # 计算不同用户的 co-rated item的稀疏矩阵C
    cmat = {}         # 储存稀疏矩阵cmat[u][v]的值 {u: {v, int}, ...}
    num = {}          # 储存每一个用户有过正反馈的物品集合的总数{user1:int, user2:int, user3:int,...}
    for item, users in item_users.items(): # 每一个item循环下，循环users元素
        for u in users:    # 由于每个item下的user是不重复的，item中每一个users都是对该item有过正正反馈，则N(user)计数+1
            if u not in num.keys():
                num[u] = 0
            num[u] += 1
            if u not in cmat.keys():     # 初始化新增到C中的u: cmat[u]
                cmat[u] = {}
            for v in users:           # users中出u之外的其他user,并计算u和v的相似度
                if u == v:
                    continue          # 跳到最近所在循环的开头
                if v not in cmat[u].keys():
                    cmat[u][v] = 0
                cmat[u][v] += 1
    
    # 计算最终的相似度矩阵 sim[u][v] 格式 {u: {v: sim(u,v)}, ...}
    sim = {}
    for u, related_users in cmat.items():
        sim[u] = {}                   # 不能少，确保sim[u]也是字典才能进一步sim[u][v]
        for v, cuv in related_users.items():
            sim[u][v] = cuv / math.sqrt(num[u] * num[v])
            
    # 按照相似度排序并获取推荐接口函数
    def GetRecommendation(user):
        rank = {}                                         # 待推荐列表  {item1:rank1, item2:rank2,...}
        # 用户见过的item列表 [item1, item2, item3, ...]
        interacted_items = set(train[user])
        # 根据相似度对user与其他user之间的相似性进行排序 {u:{v, sim(u,v)}} --> {v, sim(u,v)}
        sim_users = sim[user]
        # 对相似性用户按相似度排序并取topN的用户: simuv作为排序参考，降序
        sim_users_sorted = sorted(sim_users.items(), key=lambda x: x[1], reverse=True)[:K]
        # 根据相似度高的用户的列表对user进行推荐（去掉user见过的item）
        for v, simuv in sim_users_sorted:
            for item in train[v]:                        # train[v]是用户v的item列表
                if item not in interacted_items:  # 只处理未见过的item
                    if item not in rank.keys():          # 判断item是否已在推荐列表中
                        rank[item] = 0                   # 不在，则加入到推荐列表
                    rank[item] += sim[user][v] * 1       # 并将相似度传入rank[item]字典中(此处权重皆为1，因为train中只有user和item)
        # 对rank字典排序，获得 topN 对应的 item
        rank_sorted = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:N]
        '''若只保存 item，不需要#item: [item1, item2, item3, ...]'''
#         rank_sorted = list(map(lambda x: x[0], rank_sorted))
        # 返回值是列表
        return rank_sorted
    
    return GetRecommendation

In [9]:
# 计算最终的相似度矩阵 sim[u][v] 也可以采用下述代码，更合适
'''
    for u in sim:
        for v in sim[u]:
            C[u][v] /= math.sqrt(num[u] * num[v])
            # C[u][v] 即计算的相似度
'''

'\n    for u in sim:\n        for v in sim[u]:\n            C[u][v] /= math.sqrt(num[u] * num[v])\n            # C[u][v] 即计算的相似度\n'

### 4. UserIIF 基于用户的协同过滤推荐的改进版（余弦相似度）
步骤：
- 同3

改进之处：
- 计算用户相似度的公式有所改变
- 改进原因：两个用户对冷门物品采用过同样的行为更能说明他们兴趣的相似度
- 改进方式：惩罚了用户u和用户v共同兴趣列表中热门物品对他们的相似度的影响
- 具体公式见 《推荐系统实践》 P49


In [10]:
# 4. 基于用户余弦相似度的推荐改进版
def UserIIF(train, K, N):
    '''
    :params: train, 训练数据集
    :params: K, 超参数，设置取TopK相似用户数目
    :params: N, 超参数，设置取TopN推荐物品数目
    :return: GetRecommendation, 推荐接口函数
    '''
    # 计算 item->user 的倒排表 {item, set(user1, user4, user7), ...}
    item_users = {}
    for user in train.keys():             # 遍历用户
        for item in train[user]:          # 遍历该循环内user对应的所有items
            if item not in item_users.keys():
                item_users[item] = set()  # item_users新增的item，先设置对应user集合为空集合
            item_users[item].add(user) # 设置为空集合后，将该新增的item对应的user添加到用户集合中
    
    # 计算不同用户的 co-rated item的稀疏矩阵C
    cmat = {}         # 储存稀疏矩阵cmat[u][v]的值 {u: {v, int}, ...}
    num = {}          # 储存每一个用户有过正反馈的物品集合的总数{user1:int, user2:int, user3:int,...}
    for item, users in item_users.items(): # 每一个item循环下，循环users元素
        for u in users:    # 由于每个item下的user是不重复的，item中每一个users都是对该item有过正正反馈，则N(user)计数+1
            if u not in num.keys():
                num[u] = 0
            num[u] += 1
            if u not in cmat.keys():     # 初始化新增到C中的u: cmat[u]
                cmat[u] = {}
            for v in users:           # users中出u之外的其他user,并计算u和v的相似度
                if u == v:
                    continue          # 跳到最近所在循环的开头
                if v not in cmat[u].keys():
                    cmat[u][v] = 0
#                 cmat[u][v] += 1     #  改进的地方,改为下一行所示公式
                cmat[u][v] += (1 / math.log(1 + len(users)))
    
    # 计算最终的相似度矩阵 sim[u][v] 格式 {u: {v: sim(u,v)}, ...}
    sim = {}
    for u, related_users in cmat.items():
        sim[u] = {}                   # 不能少，确保sim[u]也是字典才能进一步sim[u][v]
        for v, cuv in related_users.items():
            sim[u][v] = cuv / math.sqrt(num[u] * num[v])
            
    # 按照相似度排序并获取推荐接口函数
    def GetRecommendation(user):
        rank = {}                                         # 待推荐列表  {item1:rank1, item2:rank2,...}
        # 用户见过的item列表 [item1, item2, item3, ...]
        interacted_items = set(train[user])
        # 根据相似度对user与其他user之间的相似性进行排序 {u:{v, sim(u,v)}} --> {v, sim(u,v)}
        sim_users = sim[user]
        # 对相似性用户按相似度排序并取topN的用户: simuv作为排序参考，降序
        sim_users_sorted = sorted(sim_users.items(), key=lambda x: x[1], reverse=True)[:K]
        # 根据相似度高的用户的列表对user进行推荐（去掉user见过的item）
        for v, simuv in sim_users_sorted:
            for item in train[v]:                        # train[v]是用户v的item列表
                if item not in interacted_items:  # 只处理未见过的item
                    if item not in rank.keys():          # 判断item是否已在推荐列表中
                        rank[item] = 0                   # 不在，则加入到推荐列表
                    rank[item] += sim[user][v] * 1       # 并将相似度传入rank[item]字典中(此处权重皆为1，因为train中只有user和item)
        # 对rank字典排序，获得 topN 对应的 item
        rank_sorted = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:N]
#         print("Recommended items list: ", rank_sorted)
        # 返回值是列表
        return rank_sorted
        
    return GetRecommendation

## 三、测试算法
1. Random实验
2. MostPopular实验
3. UserCF实验，K=[5, 10, 20, 40, 80, 160]
4. UserIIF实验, K=80

N = 10 （top10作为推荐物品）

In [11]:
class Experiment():
    def __init__(self, K, N, fp=r'..\data\ml-1m\ratings.dat', method='UserCF'):
        '''
        :params: K, TopK相似用户的个数
        :params: N, TopN推荐物品的个数
        :params: fp, 数据文件路径
        :params: method, 推荐算法
        '''
        self.K = K
        self.N = N
        self.fp = fp
        self.method = method
        self.alg = {"Random": Random, "MostPopular": MostPopular, "UserCF": UserCF, "UserIIF": UserIIF}
        
    @timmer
    def worker(self, train, test):
        '''
        :params: train, 训练数据集
        :params: test, 测试数据集
        :return: 各指标的值
        '''
        getRecommendation = self.alg[self.method](train, self.K, self.N)
        metric = Metric(train, test, getRecommendation)
        return metric.eval()
    
    @timmer
    def run(self):
        dataset = Dataset(self.fp)
        train, test = dataset.splitData()
        metric = self.worker(train, test)
        print('Done!!') 

**示例1： Random 推荐**

In [12]:
K = 10
N = 5
random_exp = Experiment(K, N, method='Random')
random_exp.run()

Func loadData, run time: 1.415750376
Func splitData, run time: 1.741130451
Metric: {'Precision': 0.92, 'Recall': 0.14, 'Coverage': 99.97, 'Popularity': 4.334407}
Func worker, run time: 17.727695567
Done!!
Func run, run time: 20.989055712000003


**示例2： MostPopular 推荐**

In [13]:
K = 80
N = 10
random_exp = Experiment(K, N, method='MostPopular')
random_exp.run()

Func loadData, run time: 1.4029202209999987
Func splitData, run time: 1.6466635789999984
Metric: {'Precision': 18.42, 'Recall': 5.55, 'Coverage': 2.04, 'Popularity': 7.647028}
Func worker, run time: 8.892803098000002
Done!!
Func run, run time: 12.047494272000002


**示例3： UserCF 推荐**

In [14]:
K = 80
N = 5
random_exp = Experiment(K, N, method='UserCF')
random_exp.run()

Func loadData, run time: 1.4449662419999996
Func splitData, run time: 1.6503386029999945
Metric: {'Precision': 26.13, 'Recall': 3.94, 'Coverage': 41.41, 'Popularity': 6.816413}
Func worker, run time: 237.029874071
Done!!
Func run, run time: 240.23973838


**示例4： UserIIF 推荐**

In [ ]:
K = 80
N = 5
random_exp = Experiment(K, N, method='UserIIF')
random_exp.run()

Func loadData, run time: 1.729870428999675
Func splitData, run time: 2.2546620460002487


In [16]:
# 不知道为什么K折之后的数据集运行起来有问题
# 